<font size=+2 color="indigo"><center><b>COVID19 Tweets - Sentiment & Geographical Analysis</b></center></font>

![](https://pbs.twimg.com/profile_images/1308010958862905345/-SGZioPb_400x400.jpg)

# Introduction

Hello readers, I am going to perform analysis on peoples tweets about COVID throughout the world.I hope i will make you realize about sentiments and emotions spread out by people in terms of tweets in twitter.This kernel will be a mixture of geographical positions and sentiment analysis of tweets.Along with EDA,sentiment analysis, deep learning model is performed to predict people's real tone of voice.After modelling,i will showcase some of post prediction visulaizations which would give more insights and results.

**Kernel Main Agenda:**

*  Data
*  EDA
*  Geographical tweets
*  Modeling & Predictions - Sentiment Analysis
*  Post Prediction Visualization


<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Table of content</h3>

* [1. Library](#1)
* [2. Data](#2)
* [3. Location](#3)
* [4. Tweet Series](#4)
* [5. Tweet Source](#5)
* [6. Hashtag](#6)
* [7. Mentions](#7)
* [8. Average Length](#8)  
* [9. Most Used Words](#9)
* [10. N-Gram](#10)
* [11. Modeling](#11)
* [12. Prediction](#12)
* [13. Angry Bird vs Happy Bird](#13)
* [14. Sentiment length](#14)
* [15. World Emotions](#15)
* [16. Top Countries - Emotional Bird](#16)
* [17. Expand sentiment classes](#17)
* [18. Vietnamese sentiment analysis](#18)

<a id="1"></a>
    
<font size="+2" color="indigo"><b>1. Library</b></font><br>

Import necessary libraries - Numpy,Ploltly,Sklearn,NLTK,Tensorflow

In [ ]:
import numpy as np 
import pandas as pd 
import os
import itertools

#plots
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from PIL import Image
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from nltk.util import ngrams


from sklearn import preprocessing
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split

import re
from collections import Counter

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords

import requests
import json

<a id="2"></a>
<font size="+2" color="indigo"><b>2. Data</b></font><br>

The main data  for this analysis and sentiment prediction is based on [Gabriel dataset](https://www.kaggle.com/gpreda/covid19-tweets)

[Kazanova dataset](https://www.kaggle.com/kazanova/sentiment140) with 1.6 milllion tweets will be used for modeling.This dataset has target feature - sentiment as 0 & 4 (0 - Negative , 4 - Positive) which will be relabeled as (0 & 1)

We wil be scaling positive and negative emotions in our covid tweet data through predictions.

**Postive emotions** - Hope,Pride,Interest,Joy,Satisfaction,Happy etc <br>
**Negative emotions** - Fear,Anger,Discust,Sadness,Rude etc

In [ ]:
covid=pd.read_csv('/kaggle/input/covid19-tweets/covid19_tweets.csv')

sentiment=pd.read_csv('../input/sentiment140/training.1600000.processed.noemoticon.csv',encoding = 'latin',header=None,names=['target','id',
                                                                                                                             'time','query','usr','text'])
# Useful for code matching with countries - Plotly Chlorepeth MAP
country_code=pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv') 

# Below files are used for data cleaning - Retrieving countries with help of city codes/state codes...(optional)
state = json.loads(requests.get("https://raw.githubusercontent.com/praneshsaminathan/country-state-city/master/states.json").text)
country=json.loads(requests.get("https://raw.githubusercontent.com/praneshsaminathan/country-state-city/master/countries.json").text)
city=json.loads(requests.get("https://raw.githubusercontent.com/praneshsaminathan/country-state-city/master/cities.json").text)
us_state_code=pd.read_csv('https://worldpopulationreview.com/static/states/abbr-name.csv',names=['state_code','state'])


# All above mentioned data are functioned and retrieved in below file to get valid country name- without_country_name
#without_country_name=pd.read_csv('../input/country-tweet/without_country_name.csv',low_memory=False)
without_country_name = pd.DataFrame([state,country,city])

covid.head(3)

In [ ]:
print('Total tweets in this data: {}'.format(covid.shape[0]))
print('Total Unique Users in this data: {}'.format(covid['user_name'].nunique()) )

<a id="3"></a>
<font size="+2" color="indigo"><b>3. Location</b></font><br>

Most tweeted countries on COVID matters. 

(Given that the location fields is  a mixture and noisy.I have done some data cleaning and fitted data into 'without_country_name.csv')

In [ ]:
covid['country_name']=covid['user_location'].str.split(',').str[-1]
covid['only_date']=pd.to_datetime(covid['date']).dt.date

#Keeping countries with valid country name

with_country_name=covid[covid['country_name'].isin(list(country_code['COUNTRY']))]
with_country_name['filtered_name']=covid['country_name']

#Without valid country name is programmed below


It took a few minutes

In [ ]:
# country=pd.DataFrame(country['countries'])
# states =pd.DataFrame(state['states'])
# city=pd.DataFrame(city['cities'])

# all_world=pd.merge(city,states,left_on="state_id",right_on="id",how="left")
# all_world=pd.merge(all_world,country,left_on="country_id",right_on="id",how="left")
# all_world=pd.merge(all_world,us_state_code,left_on="name_y",right_on="state",how="left")

# temp_df=all_world[['name_x','name']].rename(columns={'name_x':'place'})
# temp_df=temp_df.append(all_world[['name_y','name']].rename(columns={'name_y':'place'}))
# temp_df=temp_df.append(all_world[['sortname','name']].rename(columns={'sortname':'place'}))
# temp_df=temp_df.append(all_world[['state_code','name']].rename(columns={'state_code':'place'}))
# temp_df=temp_df.drop_duplicates()
# temp_df.shape


# dict_count=dict({'USA':'United States','UK':'United Kingdom','भारत':'India','British Columbia':'Canada','Deutschland':'Germany','Jammu And Kashmir':'India',
#                  'ON':'Canada','DC':'United States','UAE':'United Arab Emirates','hyderabad':'India','New York City':'United States','Everywhere':'United States',
#                  'Republic of the Philippines':'Philippines','Africa':'South Africa','WORLDWIDE':'United States','Washington DC':'United States','mumbai':'India',
#                  'INDIA':'India','Worldwide':'United States','North America':'United States','Washington DC & Virginia':'United States','PRC':'China',
#                  'San Francisco Bay Area':'United States','America':'United States','BC':'United Kingdom','BENGALURU':'India','#AFRICA #MENA':'South Africa',
#                  'online':'India','Québec':'Canada','Earth':'United States','Canberra':'Australia','Europe':'Canada','World':'India','Northern Ireland':'United Kingdom',
#                  'Sun Valley Idaho':'United States','Hong Kong':'China','Sydney':'Australia','NYC':'United States','New South Wales':'Australia','D.C.':'United States','The Netherlands':'Netherlands','Global':'United States','Planet Earth':'India','Bangalore':'India','U.S.':'United States',
#                  'CANADA':'Canada','Nig':'Nigeria','Western Australia':'Australia','The seventh house':'United States','Ngovhela Mahunguni':'Russia','Kashmir':'India','Etats-Unis':'United States','Mumbai | Kolkata':'India','VadaChennai':'India','SoCal':'United States','Sverige':'Sweden','Victoria BC':'Canada','Kingdom of Saudi Arabia':'Saudi Arabia','worldwide':'United States','Koramangala':'India','East Legon':'Ghana','india':'India','Silicon Valley':'United States','BHARAT':'India','Melbourne but I tour worldwide':'Australia','Remote':'United States','New Delhi.':'India','Cape Town':'South Africa','Nigeria.':'Nigeria','Netherlands':'Netherlands','Kamloops':'Canada','EU Citizen':'Canada','SF Bay Area':'United States','South Florida':'United States','Nova Scotia':'Canada','AB':'Canada','City of London':'United Kingdom','NOIDA':'India','NEW DELHI':'India','Lancashire and Europe':'United Kingdom','Washington D.C.':'United States','Middle East':'Saudi Arabia','Quezon City':'Philippines','@CapricornFMNews':'Russia','South Australia':'Australia','India.':'India','International':'United States','Kashmir & Ladakh':'India','WorldWide':'India','Ca':'united states','MontrÃ©al':'Canada','Asia':'India','CHINA':'China','World Wide':'India','Northern California':'United States','uk':'United Kingdom','Kuala Lumpur':'Singapore','Global Citizen':'United States','Johannesburg South Africa':'South Africa','J&K':'India','Australia ðŸ‡¦ðŸ‡º':'Australia','Abuja':'Nigeria','Makati City':'Philippines','Detroit-Northville-St. Heights':'United States','South Africa- Gauteng':'South Africa','Southern California':'United States','EspaÃ±a':'Spain',
#                  'California USA ðŸ‡ºðŸ‡¸':'United States','United States of America':'United States','West of Minsk':'Belarus',
#                  'Ontario Canada':'Canada','Greater Vancouver':'Canada','Chicago/Washington D.C.':'United States','California USA 🇺🇸':'United States','U.S.A.':'United States','Macau S.A.R.':'China','MontrÃ©al':'Spain','EspaÃ±a':'Spain','Montserrat':'United Kingdom','California USA ðŸ‡ºðŸ‡¸':'United States','TÃ¼rkiye':'Turkey','united states':'United States','Australia ðŸ‡¦ðŸ‡º':'Australia','Islamabad':'Pakistan',
#                 'Netherlands The':'Netherlands','Australia 🇦🇺':'Australia' ,'Montréal':'Canada','España':'Spain','Türkiye':'Türkey','East of England':'United Kingdom','NY USA':'United States','Waikato New Zealand':'New Zealand','Mexico City':'Mexico','West Yorkshire':'United Kingdom','NIGERIA':'Nigeria','London UK':'United Kingdom','Ngunnawal Country Aka Canberra':'Australia','Blackburn with Darwen':'United Kingdom','JHB':'South Africa','New England':'United Kingdom','UK.':'United Kingdom','Odisha(India)':'India','london':'United Kingdom','B.C.':'United Kingdom','Mysore and BERLIN':'India','Appalachia':'United States','Philly':'United States','criminal australia':'Australia','EU':'Spain','New York Metropolitan Area':'United States','U.K.':'United States','Islamic Republic of Iran':'Iran','Yorkshire and The Humber':'United Kingdom',
#                  'Northwest Indiana':'Indiana','Kenya.':'Kenya','Nairobi Kenya':'Kenya','Abu Dhabi':'Saudi Arabia'})

# #dict_count.keys()
# dict_country= pd.DataFrame(dict_count.items(), columns=['mislabel', 'correct_label'])
# # dict_country['mislabel']=dict_count.keys()
# # dict_country['correct_label']=dict_count.values()


# def manual_fix(cnt):
#     if cnt in list(dict_country['mislabel']):
#         return dict_country[dict_country['mislabel']==cnt]['correct_label'].to_string().split("  ")[-1]  
#     else:
#         return cnt


# def get_country(x):
#     if type(x) is str:
#         x = x.replace('\D+', '')
#         print(x)
#         if(len(temp_df[temp_df['place']==x.strip()]['name'])>0):
#             return temp_df[temp_df['place']==x.strip()]['name'][:1].to_string().split('  ')[-1]
#         else:
#             return manual_fix(x.strip())
#     else:
#         return x

# without_country_name['filtered_name']=covid['country_name'].apply(lambda x:get_country(x))
# without_country_name.to_csv('without_country_name.csv',index=False)

In [ ]:
tweet_df=with_country_name.append(without_country_name)
tweet_state_count=tweet_df['filtered_name'].value_counts().to_frame().reset_index().rename(columns={'index':'country','filtered_name':'count'})
all_tweet_location=pd.merge(tweet_state_count,country_code[['COUNTRY','CODE']],left_on="country",right_on="COUNTRY",how="left")
all_tweet_location=all_tweet_location[all_tweet_location['COUNTRY'].notnull()]
all_tweet_location[['COUNTRY','count']].head(5)

In [ ]:
fig = go.Figure(go.Bar(
    x=all_tweet_location['COUNTRY'][:10],y=all_tweet_location['count'][:10],
    marker={'color': all_tweet_location['count'][:10], 
    'colorscale': 'blues'},  
    text=all_tweet_location['count'][:10],
    textposition = "outside",
))
fig.update_layout(title_text='Top Countries with most tweets',xaxis_title="Countries",
                  yaxis_title="Number of Tweets ",template="plotly_dark",height=700,title_x=0.5)

fig.show()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = all_tweet_location['CODE'],
    z = all_tweet_location['count'],
    text = all_tweet_location['COUNTRY'],
    colorscale = 'rainbow', 
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = '# of Tweets',
))

fig.update_layout(
    title_text='Tweets over the world - ({} - {}) '.format(covid['only_date'].sort_values()[0].strftime("%d/%m/%Y"),
                                                       covid['only_date'].sort_values().iloc[-1].strftime("%d/%m/%Y")),title_x=0.5,
    geo=dict(
        showframe=True,
        showcoastlines=False,
        
        projection_type='equirectangular',
    )
)


fig.show()

**US,India,UK,Canada & Australia** have produced more tweets.

<a id="4"></a>
<font size="+2" color="indigo"><b>4. Tweet Series</b></font><br>

How many tweets were posted everyday?

In [ ]:
covid['tweet_date']=pd.to_datetime(covid['date']).dt.date
tweet_date=covid['tweet_date'].value_counts().to_frame().reset_index().rename(columns={'index':'date','tweet_date':'count'})
tweet_date['date']=pd.to_datetime(tweet_date['date'])
tweet_date=tweet_date.sort_values('date',ascending=False)
tweet_date.head(5)

In [ ]:
fig=go.Figure(go.Scatter(x=tweet_date['date'],
                                y=tweet_date['count'],
                               mode='markers+lines',
                               name="Submissions",
                               marker_color='dodgerblue'))

fig.update_layout(
    title_text='Tweets per Day : ({} - {}) '.format(covid['only_date'].sort_values()[0].strftime("%d/%m/%Y"),
                                                       covid['only_date'].sort_values().iloc[-1].strftime("%d/%m/%Y")),template="plotly_dark",
    title_x=0.5)

fig.show()


More tweets were made on july 25 (weekend)

<a id="5"></a>
<font size="+2" color="indigo"><b>5. Tweet Sources</b></font><br>

What device/source did people use to tweet?

In [ ]:
source_df=covid['source'].value_counts().to_frame().reset_index().rename(columns={'index':'source','source':'count'})[:15]
source_df.head(5)

In [ ]:
fig = go.Figure(go.Bar(
    x=source_df['source'],y=source_df['count'],
    marker={'color': source_df['count'], 
    'colorscale': 'blues'},  
    text=source_df['count'],
    textposition = "outside",
))

fig.update_layout(title_text='Top Sources ',xaxis_title="Sources",yaxis_title="Count ",
                  template="plotly_dark",title_x=0.5)
fig.show()


This is quite expected as there are more **Web** users,we are observing high count on it.Next comes the **Android & Iphone** users which stands next to web apps

<a id="6"></a>

<font size="+2" color="indigo"><b>6. Hashtag</b></font><br>

What hashtags has been viral/most used in covid tweets?

In [ ]:
def find_hash(text):
    line=re.findall(r'(?<=#)\w+',text)
    return " ".join(line)
covid['hash']=covid['text'].apply(lambda x:find_hash(x))

In [ ]:
hastags=list(covid[(covid['hash'].notnull())&(covid['hash']!="")]['hash'])
hastags = [each_string.lower() for each_string in hastags]
hash_df=dict(Counter(hastags))
top_hash_df=pd.DataFrame(list(hash_df.items()),columns = ['word','count']).sort_values('count',ascending=False)[:20]
top_hash_df.head(4)

In [ ]:
fig = go.Figure(go.Bar(
    x=top_hash_df['word'],y=top_hash_df['count'],
    marker={'color': top_hash_df['count'], 
    'colorscale': 'blues'},  
    text=top_hash_df['count'],
    textposition = "outside",
))
fig.update_layout(title_text='Top Trended Hastags',xaxis_title="Hashtags ",
                  yaxis_title="Number of Tags ",template="plotly_dark",height=700,title_x=0.5)
fig.show()


<a id="7"></a>

<font size="+2" color="indigo"><b>7. Mentions</b></font><br>

Let us find most mentioned user or organization in covid tweets

In [ ]:
def find_at(text):
    line=re.findall(r'(?<=@)\w+',text)
    return " ".join(line)
covid['mention']=covid['text'].apply(lambda x:find_at(x))


In [ ]:
mentions=list(covid[(covid['mention'].notnull())&(covid['mention']!="")]['mention'])
mentions = [each_string.lower().split() for each_string in mentions]
mentions=list(itertools.chain.from_iterable(mentions))
mention_df=dict(Counter(mentions))
top_mention_df=pd.DataFrame(list(mention_df.items()),columns = ['word','count']).sort_values('count',ascending=False)[:20]
top_mention_df.head(10)

In [ ]:
fig = go.Figure(go.Bar(
    x=top_mention_df['word'],y=top_mention_df['count'],
    marker={'color': top_mention_df['count'], 
    'colorscale': 'blues'},  
    text=top_mention_df['count'],
    textposition = "outside",
))

fig.update_layout(title_text='Top Trended Hastags ',xaxis_title="Hashtags",
                  yaxis_title="Number of Tags ",template="plotly_dark",height=700,title_x=0.5)
fig.show()


We observed that most tweeted countries are **US,India and UK**.As a result of them,here we could see **Trump,Biden,Boris,Modi (Top Leaders)** are tagged.Apart from them,**WHO** has paid attention to whole word.youtube is in the game too.

<a id="8"></a> 
<font size="+2" color="indigo"><b>8. Average Length</b></font><br>

What is the average length for a covid tweet?

In [ ]:
def remove_tag(string):
    text=re.sub('<.*?>','',string)
    return text
def remove_mention(text):
    line=re.sub(r'@\w+','',text)
    return line
def remove_hash(text):
    line=re.sub(r'#\w+','',text)
    return line

def remove_newline(string):
    text=re.sub('\n','',string)
    return text
def remove_url(string): 
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',string)
    return text
def remove_number(text):
    line=re.sub(r'[0-9]+','',text)
    return line
def remove_punct(text):
    line = re.sub(r'[!"\$%&\'()*+,\-.\/:;=#@?\[\\\]^_`{|}~]*','',text)
    #string="".join(line)
    return line
def text_strip(string):
    line=re.sub('\s{2,}', ' ', string.strip())
    return line
def remove_thi_amp_ha_words(string):
    line=re.sub(r'\bamp\b|\bthi\b|\bha\b',' ',string)
    return line

In [ ]:
covid['refine_text']=covid['text'].str.lower()
covid['refine_text']=covid['refine_text'].apply(lambda x:remove_tag(str(x)))
covid['refine_text']=covid['refine_text'].apply(lambda x:remove_mention(str(x)))
covid['refine_text']=covid['refine_text'].apply(lambda x:remove_hash(str(x)))
covid['refine_text']=covid['refine_text'].apply(lambda x:remove_newline(x))
covid['refine_text']=covid['refine_text'].apply(lambda x:remove_url(x))
covid['refine_text']=covid['refine_text'].apply(lambda x:remove_number(x))
covid['refine_text']=covid['refine_text'].apply(lambda x:remove_punct(x))
covid['refine_text']=covid['refine_text'].apply(lambda x:remove_thi_amp_ha_words(x))
covid['refine_text']=covid['refine_text'].apply(lambda x:text_strip(x))

covid['text_length']=covid['refine_text'].str.split().map(lambda x: len(x))


In [ ]:
fig = go.Figure(data=go.Violin(y=covid['text_length'], box_visible=True, line_color='black',
                               meanline_visible=True, fillcolor='royalblue ', opacity=0.6,
                               x0='Tweet Text Length '))

fig.update_layout(yaxis_zeroline=False,title="Distribution of Text length ",template='ggplot2')
fig.show()

Average length of tweets lies around **14** and median lies around **15**.And interquartiles lies between **11 & 18**.There is not much significant difference between mean and median which displays that people are more biased towrds this particular length.

<a id="9"></a>

<font size="+2" color="indigo"><b>9. Most Used Words</b></font><br>

Finding the most used words from entire population of tweets

In [ ]:
fig, (ax2) = plt.subplots(1,1,figsize=[17, 10])
wordcloud2 = WordCloud(background_color='black',colormap="Blues", 
                        width=600,height=400).generate(" ".join(covid['refine_text']))

ax2.imshow(wordcloud2,interpolation='bilinear')
ax2.axis('off')
ax2.set_title('Most Used Words in Comments ',fontsize=35)


**new cases,pandemic,due to** are most used ones

<a id="10"></a>

<font size="+2" color="indigo"><b>10. N-Gram</b></font><br>

Listing below the top N-gram sequential words used in covid tweets

In [ ]:
def ngram_df(corpus,nrange,n=None):
    vec = CountVectorizer(stop_words = 'english',ngram_range=nrange).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    total_list=words_freq[:n]
    df=pd.DataFrame(total_list,columns=['text','count'])
    return df
unigram_df=ngram_df(covid['refine_text'],(1,1),20)
bigram_df=ngram_df(covid['refine_text'],(2,2),20)
trigram_df=ngram_df(covid['refine_text'],(3,3),20)


In [ ]:
fig = make_subplots(
    rows=3, cols=1,subplot_titles=("Unigram","Bigram",'Trigram'),
    specs=[[{"type": "scatter"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]
          ])

fig.add_trace(go.Bar(
    y=unigram_df['text'][::-1],
    x=unigram_df['count'][::-1],
    marker={'color': "blue"},  
    text=unigram_df['count'],
    textposition = "outside",
    orientation="h",
    name="Months",
),row=1,col=1)

fig.add_trace(go.Bar(
    y=bigram_df['text'][::-1],
    x=bigram_df['count'][::-1],
    marker={'color': "blue"},  
    text=bigram_df['count'],
     name="Days",
    textposition = "outside",
    orientation="h",
),row=2,col=1)

fig.add_trace(go.Bar(
    y=trigram_df['text'][::-1],
    x=trigram_df['count'][::-1],
    marker={'color': "blue"},  
    text=trigram_df['count'],
     name="Days",
    orientation="h",
    textposition = "outside",
),row=3,col=1)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title_text='Top N Grams',xaxis_title=" ",yaxis_title=" ", showlegend=False,title_x=0.5,
                  height=1200,template="plotly_dark")
fig.show()

* **new,people,pandemic** are most used single word.
* **cases death,tested postive and active cases** are most used bigrams.
* **cases new deaths,help slow spread,slow spread indentity** are most used trigrams

All observed words are biased towards covid detection and admission of patients.People tend to worry lot about other peoples around.

<a id="11"></a>

<font size="+2" color="indigo"><b>11. Modeling</b></font><br>

I will be using 40% of the data to train.I have optimized the text by removing noisy and unwanted characters present in tweet.

In [ ]:
sentiment=sentiment.sample(int(sentiment.shape[0]*0.4))
sentiment=sentiment[['text','target']]
sentiment['emotion']=np.where(sentiment['target']==0,'negative',np.where(sentiment['target']==2,'neutral',np.where(sentiment['target']==4,'postitive',"none")))
sentiment['target']=np.where(sentiment['target']==4,1,sentiment['target'])

In [ ]:
# Optimizing text
sentiment['refine_text']=sentiment['text'].str.lower()
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:remove_tag(str(x)))
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:remove_mention(str(x)))
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:remove_hash(str(x)))
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:remove_newline(x))
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:remove_url(x))
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:remove_number(x))
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:remove_punct(x))
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:remove_thi_amp_ha_words(x))
sentiment['refine_text']=sentiment['refine_text'].apply(lambda x:text_strip(x))
sentiment['text_length']=sentiment['refine_text'].str.split().map(lambda x: len(x))

# Removing stopwords
stop_words = set(stopwords.words('english'))
sentiment['refine_text'] =  sentiment['refine_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


In [ ]:
# Setting tha randomly
train_data=sentiment[['refine_text','target']]

train_sent=np.array(train_data['refine_text'])
train_label=np.array(train_data['target'])

x_train, x_test, y_train, y_test = train_test_split(train_sent, train_label, test_size=0.20, random_state=42)

### Tuning Parametes (Change below parameters/add or remove layers to get more accurate result)

In [ ]:
# Hyper Parameters
vocab_size = 1000
embedding_dim =16 
max_length = 50
trunc_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(x_train)
training_padded = pad_sequences(training_sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

training_label=y_train
testing_label=y_test

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')

])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 1 # Edit here
history=model.fit(training_padded, training_label, epochs=num_epochs, validation_data=(testing_padded, testing_label))

We have almost hedged our model accuracy with validation accuracy.The model achieved around **75%** accuracy which is not too good or too bad to measure.Still we can tune hyper parameters and produce more good results.

<a id="12"></a>

<font size="+2" color="indigo"><b>12. Prediction</b></font><br>

In [ ]:
tweet_df['refine_text']=tweet_df['text'].str.lower()
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:remove_tag(str(x)))
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:remove_mention(str(x)))
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:remove_hash(str(x)))
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:remove_newline(x))
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:remove_url(x))
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:remove_number(x))
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:remove_punct(x))
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:remove_thi_amp_ha_words(x))
tweet_df['refine_text']=tweet_df['refine_text'].apply(lambda x:text_strip(x))
tweet_df['text_length']=tweet_df['refine_text'].str.split().map(lambda x: len(x))

In [ ]:
sen = np.array(tweet_df.refine_text)
seq = tokenizer.texts_to_sequences(sen)
padd = pad_sequences(seq, maxlen=max_length, truncating=trunc_type)
result=model.predict(padd)
validated_result=np.where(result>0.5,1,0)

In [ ]:
pred_df=pd.DataFrame({'text':tweet_df['refine_text'],'pred_sentiment':list(validated_result),'country':tweet_df['filtered_name'],'text_length':tweet_df['text_length']})
pred_df['pred_sentiment']=np.where(pred_df['pred_sentiment']>0.5,1,0)
pred_df[['text','pred_sentiment']].head(4)

Below we will observe the post prediction visualizations.It may differ slightly for every change in models.

<a id="13"></a>

<font size="+2" color="indigo"><b>13. Angry Bird vs Happy Bird</b></font><br>

We will find most frequent used words on Negative tweet & Positive tweet.

In [ ]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get('https://banner2.cleanpng.com/20180723/vvy/kisspng-computer-icons-clip-art-twitter-logo-vector-5b5693f7952128.7797517715324006316109.jpg')
bird = np.array(Image.open(BytesIO(response.content)))

# d = '../input/twitter/'
# bird = np.array(Image.open(d + 'twitter_mask.png'))
fig, (ax2, ax3) = plt.subplots(1, 2, figsize=[30, 15])
wordcloud2 = WordCloud( background_color='white',mask=bird,colormap="Reds",
                        width=600,
                        height=400).generate(" ".join(pred_df[pred_df['pred_sentiment']==0]['text']))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Negative Sentiment',fontsize=35);

wordcloud3 = WordCloud( background_color='white',mask=bird,colormap="Greens",
                        width=600,
                        height=400).generate(" ".join(pred_df[pred_df['pred_sentiment']==1]['text']))
ax3.imshow(wordcloud3)
ax3.axis('off')
ax3.set_title('Positive Sentiment',fontsize=35);

<a id="14"></a>
<font size="+2" color="indigo"><b>14. Sentiment length</b></font><br>

In [ ]:
print("Average length of Positive Sentiment tweets : {} ".format(round(pred_df[pred_df['pred_sentiment']==1]['text_length'].mean(),2)))
print("Average length of Negative Sentiment tweets : {} ".format(round(pred_df[pred_df['pred_sentiment']==0]['text_length'].mean(),2)))

In [ ]:
fig = go.Figure()

fig.add_trace(go.Violin(y=pred_df[pred_df['pred_sentiment']==1]['text_length'], box_visible=False, line_color='black',
                               meanline_visible=True, fillcolor='green', opacity=0.6,name="Positive",
                               x0='Positive')
             )

fig.add_trace(go.Violin(y=pred_df[pred_df['pred_sentiment']==0]['text_length'], box_visible=False, line_color='black',
                               meanline_visible=True, fillcolor='red', opacity=0.6,name="Negative",
                               x0='Negative')
             )

fig.update_traces(box_visible=False, meanline_visible=True)
fig.update_layout(title_text="Violin - Tweet Length ",
                  title_x=0.5)

fig.show()


Average words for positive is around 13 whereas for negative is around 15.People tend to type long when they are in negative mode.

<a id="15"></a>
<font size="+2" color="indigo"><b>15. World Emotions</b></font><br>

Let us look at heatmap of positive and negative sentiment of each countries in the world.

In [ ]:
all_pos_country=pred_df[pred_df['pred_sentiment']==1]['country'].value_counts().reset_index().rename(columns={'index':'country','country':'count'})

all_pos_country_df=pd.merge(all_pos_country,country_code[['COUNTRY','CODE']],left_on="country",right_on="COUNTRY",how="left")
all_pos_country_df=all_pos_country_df[all_pos_country_df['COUNTRY'].notnull()]

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = all_pos_country_df['CODE'],
    z = all_pos_country_df['count'],
    text = all_pos_country_df['COUNTRY'],
    colorscale = 'greens', 
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = '# of Tweets ',
))

fig.update_layout(
    title_text='Tweets over the world - ({} - {})'.format(covid['only_date'].sort_values()[0].strftime("%d/%m/%Y"),
                                                       covid['only_date'].sort_values().iloc[-1].strftime("%d/%m/%Y")),title_x=0.5,
    
    geo=dict(
        showframe=True,
        showcoastlines=False,
        projection_type='equirectangular',
    )
)

fig.show()


### Positive Sentiment counts

In [ ]:
all_pos_country_df[['country','count']][:5]

In [ ]:
all_neg_country=pred_df[pred_df['pred_sentiment']==0]['country'].value_counts().reset_index().rename(columns={'index':'country','country':'count'})

all_neg_country_df=pd.merge(all_neg_country,country_code[['COUNTRY','CODE']],left_on="country",right_on="COUNTRY",how="left")
all_neg_country_df=all_neg_country_df[all_neg_country_df['COUNTRY'].notnull()]

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = all_neg_country_df['CODE'],
    z = all_neg_country_df['count'],
    text = all_neg_country_df['COUNTRY'],
    colorscale = 'reds',  
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = '# of Tweets',
))

fig.update_layout(
    title_text='Tweets over the world - ({} - {}) '.format(covid['only_date'].sort_values()[0].strftime("%d/%m/%Y"),
                                covid['only_date'].sort_values().iloc[-1].strftime("%d/%m/%Y")),title_x=0.5,
    geo=dict(
        showframe=True,
        showcoastlines=False,
        projection_type='equirectangular',
    )
)

fig.show()


### Negative Sentiment counts

In [ ]:
all_neg_country_df[['country','count']][:5]

Comparitive to positive sentiments,negatives are low in numbers.This is good insight which displays people are not hatred always instead handling this pandemic with good positive gesture

<a id="16"></a>
<font size="+2" color="indigo"><b>16. Top Countries - Emotional Bird</b></font><br>

Now we will see the most tweeted countries positive and negative emotional words. US,India & UK are the most involved countries in twitter tweeting on CORONA.  

In [ ]:
fig, (ax2, ax3) = plt.subplots(1, 2, figsize=[30, 15])
wordcloud2 = WordCloud( background_color='white',mask=bird,colormap="Reds",
                        width=600,
                        height=400).generate(" ".join(pred_df[(pred_df['pred_sentiment']==0)&(pred_df['country']=='United States')]['text']))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Negative US Bird',fontsize=35);

wordcloud3 = WordCloud( background_color='white',mask=bird,colormap="Greens",
                        width=600,
                        height=400).generate(" ".join(pred_df[(pred_df['pred_sentiment']==1)&(pred_df['country']=='United States')]['text']))
ax3.imshow(wordcloud3)
ax3.axis('off')
ax3.set_title('Positive US Bird',fontsize=35);

In [ ]:
fig, (ax2, ax3) = plt.subplots(1, 2, figsize=[30, 15])
wordcloud2 = WordCloud( background_color='white',mask=bird,colormap="Reds",
                        width=600,
                        height=400).generate(" ".join(pred_df[(pred_df['pred_sentiment']==0)&(pred_df['country']=='India')]['text']))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Negative India Bird ',fontsize=35);

wordcloud3 = WordCloud( background_color='white',mask=bird,colormap="Greens",
                        width=600,
                        height=400).generate(" ".join(pred_df[(pred_df['pred_sentiment']==1)&(pred_df['country']=='India')]['text']))
ax3.imshow(wordcloud3)
ax3.axis('off')
ax3.set_title('Positive India Bird ',fontsize=35);

In [ ]:
fig, (ax2, ax3) = plt.subplots(1, 2, figsize=[30, 15])
wordcloud2 = WordCloud( background_color='white',mask=bird,colormap="Reds",
                        width=600,
                        height=400).generate(" ".join(pred_df[(pred_df['pred_sentiment']==0)&(pred_df['country']=='United Kingdom')]['text']))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Negative UK Bird ',fontsize=35);

wordcloud3 = WordCloud( background_color='white',mask=bird,colormap="Greens",
                        width=600,
                        height=400).generate(" ".join(pred_df[(pred_df['pred_sentiment']==1)&(pred_df['country']=='United Kingdom')]['text']))
ax3.imshow(wordcloud3)
ax3.axis('off')
ax3.set_title('Positive UK Bird ',fontsize=35);

<a id="17"></a>
<font size="+2" color="indigo"><b>17. Homework</b></font><br>

**In covid-19-nlp-text-classification we have 6 classes, but our model only predict two classes. So all you will re-build model for 6 classes and predict this dataset again.**

<a id="18"></a>
<font size="+2" color="indigo"><b>18. Vietnamese Sentiment Analyst</b></font><br>

In [ ]:
vn_sentiment=pd.read_csv('../input/vietnamese-sentiment/data - data.csv',usecols=["comment","label"])

In [ ]:
vn_sentiment.head(10)

In [ ]:
category = vn_sentiment['label'].unique()
category_to_id = {cate: idx for idx, cate in enumerate(category)}
id_to_category = {idx: cate for idx, cate in enumerate(category)}
print(category_to_id)
print(id_to_category)

In [ ]:
data_label = vn_sentiment['label']
data_label = pd.DataFrame(data_label, columns=['label']).groupby('label').size()
data_label.plot.pie(figsize=(15, 15), autopct="%.2f%%", fontsize=12)

In here we use transformer model to tokenize word

In [ ]:
!pip install transformers

In here, we have two options: bert and gpt pretrained model to tokenize word.

Read data

In [ ]:
vn_sentiment.isnull().values.any()

vn_sentiment.shape

In [ ]:
reviews = []
sentences = list(vn_sentiment['comment'])
for sen in sentences:
    reviews.append(sen)

In [ ]:
reviews[10]

In [ ]:
vn_sentiment.label.unique()

In [ ]:
vn_sentiment.label.head(10)

In [ ]:
y = vn_sentiment['label']

# NEU : 0 , POS : 1 , NEG : 2
y = np.array(list(map(lambda x: 1 if x=="POS" else (2 if x == "NEG" else 0 ), y)))

In [ ]:
print(reviews[10],y[10])

Import libaries

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

### I used bert pretrained model to tokenize, you need use phobert to have a good performance

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [ ]:
print(tokenized_reviews[10])

In [ ]:
import random 

reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]
# Word Vector - Label - length of sequence.
print(reviews_with_len[0])

In [ ]:
# Shuffle random dataset
random.shuffle(reviews_with_len)

In [ ]:
reviews_with_len[0]

Once the data is shuffled, we will sort the data by the length of the reviews. To do so, we will use the sort() function of the list and will tell it that we want to sort the list with respect to the third item in the sublist i.e. the length of the review.

In [ ]:
#reviews_with_len.sort(key=lambda x: x[2])
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]
sorted_reviews_labels
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32 # You can edit here i.e 128
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

In [ ]:
import math

TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

### Build model, you need to re-build end-to end or using pretrained model as Bert, GPT-2

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 3

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)


In [ ]:
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# es = EarlyStopping(monitor='val_f1_m', mode='max', verbose=1, patience=5)
# reduce_lr = ReduceLROnPlateau(monitor='val_f1_m', factor=0.2, patience=8, min_lr=1e7)
# checkpoint = ModelCheckpoint('best_full.h5', monitor='val_f1_m', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [ ]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [ ]:
history = text_model.fit(train_data,validation_data=test_data, epochs=2)

### Visualization 
You can put a loss visualization in here

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(121)

# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.title('Model loss')

plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Pylot iou-score 
plt.subplot(122)

# Get training and test loss histories
training_loss = history.history['sparse_categorical_accuracy']
test_loss = history.history['val_sparse_categorical_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.title('Accuracy')
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.show()

### Predict

In [ ]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sample_pred_text, pad):
  encoded_sample_pred_text = tokenize_reviews(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = text_model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [ ]:
sample_pred_text = ('Bớt đùa đi, dạo này tao không còn vui tính như trước đâu.')

predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)